# Exercise 4 - Nested Parallelism

**GOAL:** The goal of this exercise is to show how to create nested tasks by calling a remote function inside of another remote function.

In this exercise, you will implement the structure of a parallel hyperparameter sweep which trains a number of models in parallel. Each model will be trained using parallel gradient computations.

### Concepts for this Exercise - Nested Remote Functions

Remote functions can call other functions. For example, consider the following.

```python
@ray.remote
def f():
    return 1

@ray.remote
def g():
    # Call f 4 times and return the resulting object IDs.
    return [f.remote() for _ in range(4)]

@ray.remote
def h():
    # Call f 4 times, block until those 4 tasks finish,
    # retrieve the results, and return the values.
    return ray.get([f.remote() for _ in range(4)])
```

Then calling `g` and `h` produces the following behavior.

```python
>>> ray.get(g.remote())
[ObjectID(b1457ba0911ae84989aae86f89409e953dd9a80e),
 ObjectID(7c14a1d13a56d8dc01e800761a66f09201104275),
 ObjectID(99763728ffc1a2c0766a2000ebabded52514e9a6),
 ObjectID(9c2f372e1933b04b2936bb6f58161285829b9914)]

>>> ray.get(h.remote())
[1, 1, 1, 1]
```

**One limitation** is that the definition of `f` must come before the definitions of `g` and `h` because as soon as `g` is defined, it will be pickled and shipped to the workers, and so if `f` hasn't been defined yet, the definition will be incomplete.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

import builtins


@ray.remote
def sum(xs):
    return builtins.sum(xs)

In [2]:
ray.init(num_cpus=9, redirect_output=True)

Waiting for redis server at 127.0.0.1:58085 to respond...
Waiting for redis server at 127.0.0.1:61938 to respond...
Starting local scheduler with the following resources: {'CPU': 9, 'GPU': 0}.

View the web UI at http://localhost:8891/notebooks/ray_ui34090.ipynb?token=6fce3b4daf6ee700f16eb460c30e8e429084d55346444591



{'local_scheduler_socket_names': ['/tmp/scheduler51370718'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store7217893', manager_name='/tmp/plasma_manager69299679', manager_port=10221)],
 'redis_address': '127.0.0.1:58085',
 'webui_url': 'http://localhost:8891/notebooks/ray_ui34090.ipynb?token=6fce3b4daf6ee700f16eb460c30e8e429084d55346444591'}

This example represents a hyperparameter sweep in which multiple models are trained in parallel. Each model training task also performs data parallel gradient computations.

**EXERCISE:** Turn `compute_gradient` and `train_model` into remote functions so that they can be executed in parallel. Inside of `train_model`, do the calls to `compute_gradient` in parallel and fetch the results using `ray.get`.

In [12]:
@ray.remote
def compute_gradient(data):
    time.sleep(0.03)
    return 1


@ray.remote
def train_model(hyperparameters):

    # EXERCISE: After you turn "compute_gradient" into a remote function,
    # you will need to call it with ".remote". The results must be retrieved
    # with "ray.get" before "sum" is called.
[[    sum.remote([compute_gradient.remote(i) for i in range(2)])]
for _ in range(10)]
    result = sum.remote(
        [sum.remote([compute_gradient.remote(j) for j in range(2)])
        for _ in range(10)])
    return result

**EXERCISE:** The code below runs 3 hyperparameter experiments. Change this to run the experiments in parallel.

In [13]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

# Run some hyperparaameter experiments.

hyperparameters = [
    {
        'learning_rate': 1e-1,
        'batch_size': 100,
    },
    {
        'learning_rate': 1e-2,
        'batch_size': 100,
    },
    {
        'learning_rate': 1e-3,
        'batch_size': 100,
    },
]

results = ray.get([train_model.remote(h) for h in hyperparameters])

end_time = time.time()
duration = end_time - start_time

Remote function __main__.train_model failed with:

Traceback (most recent call last):
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 790, in _process_task
    self._store_outputs_in_objstore(return_object_ids, outputs)
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 717, in _store_outputs_in_objstore
    self.put_object(objectids[i], outputs[i])
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 349, in put_object
    raise Exception("Calling 'put' on an ObjectID is not allowed "
Exception: Calling 'put' on an ObjectID is not allowed (similarly, returning an ObjectID from a remote function is not allowed). If you really want to do this, you can wrap the ObjectID in a list and call 'put' on it (or return it).


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.

RayGetError: Could not get objectid ObjectID(f420d9ba47fed2978384e7c5f6d38b960f2d75d4). It was created by remote function [31m__main__.train_model[39m which failed with:

Remote function [31m__main__.train_model[39m failed with:

Traceback (most recent call last):
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 790, in _process_task
    self._store_outputs_in_objstore(return_object_ids, outputs)
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 717, in _store_outputs_in_objstore
    self.put_object(objectids[i], outputs[i])
  File "/Users/alokbeniwal/Library/Python/3.6/lib/python/site-packages/ray/worker.py", line 349, in put_object
    raise Exception("Calling 'put' on an ObjectID is not allowed "
Exception: Calling 'put' on an ObjectID is not allowed (similarly, returning an ObjectID from a remote function is not allowed). If you really want to do this, you can wrap the ObjectID in a list and ca

Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:58085")
  
Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:58085")
  
Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

    

Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:58085")
  
Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:58085")
  
Remote function __main__.sum failed with:

Traceback (most recent call last):
  File "<ipython-input-11-0ba2ad0ebe88>", line 8, in sum
TypeError: unsupported operand type(s) for +: 'int' and 'common.ObjectID'


  You can inspect errors by running

    

**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [17]:
assert results == [20, 20, 20]
assert duration < 0.5, ('The experiments ran in {} seconds. This is too '
                         'slow.'.format(duration))
assert duration > 0.3, ('The experiments ran in {} seconds. This is too '
                        'fast.'.format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.36385107040405273 seconds.


**EXERCISE:** Use the UI to view the task timeline and to verify that the pattern makes sense.

In [18]:
import ray.experimental.ui as ui
ui.task_timeline()

To view fullscreen, open chrome://tracing in Google Chrome and load `/var/folders/52/pd9v2v217dqdkj574btgbn4h0000gp/T/tmpbgkqyaho.json`
